In [11]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd

In [2]:
tf.__version__

'2.0.0-alpha0'

In [3]:
@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x

In [8]:
print(square_if_positive(2))
print(square_if_positive(-2))

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)


In [25]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s

In [24]:
sum_even(tf.constant([10, 12, 15, 20]))

10
12
20


<tf.Tensor: id=358, shape=(), dtype=int32, numpy=42>

In [19]:
N_pdf = tfd.Poisson(1/10).prob(range(50))
N_pdf

<tf.Tensor: id=188, shape=(50,), dtype=float32, numpy=
array([9.0483743e-01, 9.0483747e-02, 4.5241872e-03, 1.5080623e-04,
       3.7701539e-06, 7.5403108e-08, 1.2567178e-09, 1.7953115e-11,
       2.2441393e-13, 2.4934939e-15, 2.4934913e-17, 2.2668163e-19,
       1.8890154e-21, 1.4530829e-23, 1.0379160e-25, 6.9194397e-28,
       4.3246561e-30, 2.5439226e-32, 1.4132820e-34, 7.4383232e-37,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)>

In [20]:
tf.print("aaaa", 10)

aaaa 10


In [38]:
N = 10
A = tf.sparse.SparseTensor(
    indices = [[i+1, i] for i in range(N-1)], 
    values = tf.ones(N-1),
    dense_shape = [N, N]
)

In [47]:
v0 = tf.concat([[1], tf.zeros(N-1)], axis = 0)
v0

<tf.Tensor: id=1604, shape=(10,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [55]:
v = v0[:, None]
acc = tf.zeros_like(v)
for p in N_pdf:
    v = tf.sparse.sparse_dense_matmul(A, v)
    acc += p * v
    
acc

<tf.Tensor: id=2675, shape=(10, 1), dtype=float32, numpy=
array([[0.0000000e+00],
       [9.0483743e-01],
       [9.0483747e-02],
       [4.5241872e-03],
       [1.5080623e-04],
       [3.7701539e-06],
       [7.5403108e-08],
       [1.2567178e-09],
       [1.7953115e-11],
       [2.2441393e-13]], dtype=float32)>

In [52]:
@tf.function
def evolve(v0, A, N_pdf):
    v = v0[:, None]
    acc = tf.zeros_like(v)
    for p in N_pdf:
        tf.print(p, v, acc)
        v = tf.sparse.sparse_dense_matmul(A, v)
        #acc += p * v

    return acc

v1 = evolve(v0, A, N_pdf)
v1

ValueError: Input tensor 'strided_slice:0' enters the loop with shape (10, 1), but has shape (None, 1) after one iteration. To allow the shape to vary across iterations, use the `shape_invariants` argument of tf.while_loop to specify a less-specific shape.

In [56]:
A = tf.zeros([N, N])

@tf.function
def evolve(v0, A, N_pdf):
    v = v0[:, None]
    acc = tf.zeros_like(v)
    for p in N_pdf:
        tf.print(p, v, acc)
        v = A @ v
        acc += p * v

    return acc

v1 = evolve(v0, A, N_pdf)
v1

0.90483743 [[1]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
0.0904837474 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
0.00452418718 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
0.000150806227 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
3.77015385e-06 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
7.54031078e-08 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
1.25671784e-09 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
1.79531147e-11 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
2.24413925e-13 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
2.4934939e-15 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
2.49349129e-17 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
2.26681635e-19 [[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] 

<tf.Tensor: id=2754, shape=(10, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>